# Imports

In [1]:
!sudo /bin/bash -c "(source /venv/bin/activate; pip install gspread_pandas)"

In [2]:
!sudo /bin/bash -c "(source /venv/bin/activate; pip install google-api-python-client)"

In [3]:
import gspread_pandas

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [4]:
import logging
import helpers.hdbg as hdbg
import linkedin.google_api.google_file_api as google_file_api

In [5]:
_LOG = logging.getLogger(__name__)
hdbg.init_logger(use_exec_path=True)

INFO: > cmd='/venv/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-445be7dc-05c5-416b-8cba-add9af2959c1.json'
INFO: Saving log to file '/app/linkedin/notebooks/none.log'


# Load data

In [6]:
spreadsheet_name = "sn_search5.search_export.gsheet"
spread = gspread_pandas.Spread(spreadsheet_name)
df = spread.sheet_to_df(index=None)
print(df.shape)
df.head()

(194, 32)


query  \
0                                         Search5 SN   
1  https://www.linkedin.com/sales/search/people?s...   
2  https://www.linkedin.com/sales/search/people?s...   
3  https://www.linkedin.com/sales/search/people?s...   
4  https://www.linkedin.com/sales/search/people?s...   

                  timestamp            error  \
0  2023-07-19T18:00:39.744Z  No result found   
1  2023-07-19T18:02:26.834Z                    
2  2023-07-19T18:02:26.834Z                    
3  2023-07-19T18:02:26.835Z                    
4  2023-07-19T18:02:26.835Z                    

                                          profileUrl               fullName  \
0                                                                             
1  https://www.linkedin.com/sales/lead/ACwAABgNPU...      Dr. Edgar Noumair   
2  https://www.linkedin.com/sales/lead/ACwAAAHVj4...  Maureen Cusick Thomas   
3  https://www.linkedin.com/sales/lead/ACwAAAE1Yq...             Neil Davis   
4  https://www.linkedin.com/sales/lead/ACwAAACRHS...           Michael Hess   

   firstName       lastName                                companyName  \
0                                                                        
1  Dr. Edgar        Noumair  Talal and Madiha Zein AUB Innovation Park   
2    Maureen  Cusick Thomas                             Bethesda Green   
3       Neil          Davis                           Innovation Works   
4    Michael           Hess          ABS Advisory Services Group, Inc.   

                   title companyId  ...  \
0                                   ...   
1                 Mentor  71418447  ...   
2  Innovation Lab Mentor   9287319  ...   
3        Business Mentor  11787752  ...   
4           Chairman/CEO            ...   

                                      vmid  \
0                                            
1  ACwAABgNPUkBxBAF3JxK0VPQmHkYRnrxXzAiHjY   
2  ACwAAAHVj44Brlumxgp-o7YP0oARKp_kKER4e6k   
3  ACwAAAE1YqQBZoy53iiEXT0Ol-GrNh_0U0dhvqM   
4  ACwAAACRHSQB6TCtuIxEce2hAb9lV3NM_GzNPc8   

                                  linkedInProfileUrl isPremium isOpenLink  \
0                                                                           
1  https://www.linkedin.com/in/ACwAABgNPUkBxBAF3J...     FALSE      FALSE   
2  https://www.linkedin.com/in/ACwAAAHVj44Brlumxg...      TRUE       TRUE   
3  https://www.linkedin.com/in/ACwAAAE1YqQBZoy53i...     FALSE      FALSE   
4  https://www.linkedin.com/in/ACwAAACRHSQB6TCtuI...     FALSE      FALSE   

                                    titleDescription  \
0                                                      
1                                                      
2  The Innovation Lab Mentor Program is designed ...   
3  Innovation Works aims to reduce Baltimore’s ne...   
4  Manage mergers & acquisitions, corporate and r...   

  pastExperienceCompanyName pastExperienceCompanyUrl  \
0                                                      
1                                                      
2                                                      
3                                                      
4                                                      

  pastExperienceCompanyTitle pastExperienceDate pastExperienceDuration  
0                                                                       
1                                                                       
2                                                                       
3                                                                       
4                                                                       

[5 rows x 32 columns]

# Filter data

In [7]:
# Drop errors.
prev_len = len(df)
df = df[df["error"] == ""].reset_index(drop=True)
df = df[[col for col in df.columns if col != "error"]]
print(
    f"Dropped {prev_len - len(df)} rows ({round((prev_len - len(df))*100/prev_len, 2)}%)"
)
df.head()

Dropped 1 rows (0.52%)


query  \
0  https://www.linkedin.com/sales/search/people?s...   
1  https://www.linkedin.com/sales/search/people?s...   
2  https://www.linkedin.com/sales/search/people?s...   
3  https://www.linkedin.com/sales/search/people?s...   
4  https://www.linkedin.com/sales/search/people?s...   

                  timestamp  \
0  2023-07-19T18:02:26.834Z   
1  2023-07-19T18:02:26.834Z   
2  2023-07-19T18:02:26.835Z   
3  2023-07-19T18:02:26.835Z   
4  2023-07-19T18:02:26.835Z   

                                          profileUrl               fullName  \
0  https://www.linkedin.com/sales/lead/ACwAABgNPU...      Dr. Edgar Noumair   
1  https://www.linkedin.com/sales/lead/ACwAAAHVj4...  Maureen Cusick Thomas   
2  https://www.linkedin.com/sales/lead/ACwAAAE1Yq...             Neil Davis   
3  https://www.linkedin.com/sales/lead/ACwAAACRHS...           Michael Hess   
4  https://www.linkedin.com/sales/lead/ACwAAADZA4...        Carl J. Wallace   

   firstName       lastName                                companyName  \
0  Dr. Edgar        Noumair  Talal and Madiha Zein AUB Innovation Park   
1    Maureen  Cusick Thomas                             Bethesda Green   
2       Neil          Davis                           Innovation Works   
3    Michael           Hess          ABS Advisory Services Group, Inc.   
4    Carl J.        Wallace          National Science Foundation (NSF)   

                                  title companyId  \
0                                Mentor  71418447   
1                 Innovation Lab Mentor   9287319   
2                       Business Mentor  11787752   
3                          Chairman/CEO             
4  Industry Mentor - CFO Tiami Networks      8599   

                                        companyUrl  ...  \
0  https://www.linkedin.com/sales/company/71418447  ...   
1   https://www.linkedin.com/sales/company/9287319  ...   
2  https://www.linkedin.com/sales/company/11787752  ...   
3                                                   ...   
4      https://www.linkedin.com/sales/company/8599  ...   

                                      vmid  \
0  ACwAABgNPUkBxBAF3JxK0VPQmHkYRnrxXzAiHjY   
1  ACwAAAHVj44Brlumxgp-o7YP0oARKp_kKER4e6k   
2  ACwAAAE1YqQBZoy53iiEXT0Ol-GrNh_0U0dhvqM   
3  ACwAAACRHSQB6TCtuIxEce2hAb9lV3NM_GzNPc8   
4  ACwAAADZA40Bmn2kVnOAiz0l7ix5q2sKM5nRPK8   

                                  linkedInProfileUrl isPremium isOpenLink  \
0  https://www.linkedin.com/in/ACwAABgNPUkBxBAF3J...     FALSE      FALSE   
1  https://www.linkedin.com/in/ACwAAAHVj44Brlumxg...      TRUE       TRUE   
2  https://www.linkedin.com/in/ACwAAAE1YqQBZoy53i...     FALSE      FALSE   
3  https://www.linkedin.com/in/ACwAAACRHSQB6TCtuI...     FALSE      FALSE   
4  https://www.linkedin.com/in/ACwAAADZA40Bmn2kVn...      TRUE       TRUE   

                                    titleDescription  \
0                                                      
1  The Innovation Lab Mentor Program is designed ...   
2  Innovation Works aims to reduce Baltimore’s ne...   
3  Manage mergers & acquisitions, corporate and r...   
4  Assisting clients in the NSF I CORP accelerato...   

  pastExperienceCompanyName pastExperienceCompanyUrl  \
0                                                      
1                                                      
2                                                      
3                                                      
4                                                      

  pastExperienceCompanyTitle pastExperienceDate pastExperienceDuration  
0                                                                       
1                                                                       
2                                                                       
3                                                                       
4                                                                       

[5 rows x 31 columns]

In [8]:
# Filter by location.
# The location you want to keep.
location = "Maryland"
prev_len = len(df)
df = df[df["location"].apply(lambda x: location in x)].reset_index(drop=True)
print(
    f"Dropped {prev_len - len(df)} rows ({round((prev_len - len(df))*100/prev_len, 2)}%)"
)
df.head()

Dropped 111 rows (57.51%)


query  \
0  https://www.linkedin.com/sales/search/people?s...   
1  https://www.linkedin.com/sales/search/people?s...   
2  https://www.linkedin.com/sales/search/people?s...   
3  https://www.linkedin.com/sales/search/people?s...   
4  https://www.linkedin.com/sales/search/people?s...   

                  timestamp  \
0  2023-07-19T18:02:26.835Z   
1  2023-07-19T18:02:26.835Z   
2  2023-07-19T18:02:26.835Z   
3  2023-07-19T18:02:26.835Z   
4  2023-07-19T18:02:26.836Z   

                                          profileUrl          fullName  \
0  https://www.linkedin.com/sales/lead/ACwAAAE1Yq...        Neil Davis   
1  https://www.linkedin.com/sales/lead/ACwAAACRHS...      Michael Hess   
2  https://www.linkedin.com/sales/lead/ACwAAAAQQk...  Barrie Zucal, MS   
3  https://www.linkedin.com/sales/lead/ACwAAADeWj...      Leslie Traub   
4  https://www.linkedin.com/sales/lead/ACwAAAAY9Y...   Theresa Daytner   

  firstName   lastName                        companyName  \
0      Neil      Davis                   Innovation Works   
1   Michael       Hess  ABS Advisory Services Group, Inc.   
2    Barrie  Zucal, MS                   Zucal Consulting   
3    Leslie      Traub           Inclusion Allies Network   
4   Theresa    Daytner              Conscious Venture Lab   

                                               title companyId  \
0                                    Business Mentor  11787752   
1                                       Chairman/CEO             
2              Independent Mentor, Coach, Consultant             
3                              Co-Founder and Mentor             
4  Lead Mentor | Angel Investor | Executive Coach...   3356498   

                                        companyUrl  ...  \
0  https://www.linkedin.com/sales/company/11787752  ...   
1                                                   ...   
2                                                   ...   
3                                                   ...   
4   https://www.linkedin.com/sales/company/3356498  ...   

                                      vmid  \
0  ACwAAAE1YqQBZoy53iiEXT0Ol-GrNh_0U0dhvqM   
1  ACwAAACRHSQB6TCtuIxEce2hAb9lV3NM_GzNPc8   
2  ACwAAAAQQkoBFSrQYiDk3QVKcRprtPvRSXe7ajk   
3  ACwAAADeWjEBRuyhZkIF7X7fpy6uBoxiXLQWqLY   
4  ACwAAAAY9YQBq8q_bnRNDw_10z49i6IR2gD7RKQ   

                                  linkedInProfileUrl isPremium isOpenLink  \
0  https://www.linkedin.com/in/ACwAAAE1YqQBZoy53i...     FALSE      FALSE   
1  https://www.linkedin.com/in/ACwAAACRHSQB6TCtuI...     FALSE      FALSE   
2  https://www.linkedin.com/in/ACwAAAAQQkoBFSrQYi...     FALSE      FALSE   
3  https://www.linkedin.com/in/ACwAAADeWjEBRuyhZk...     FALSE      FALSE   
4  https://www.linkedin.com/in/ACwAAAAY9YQBq8q_bn...      TRUE      FALSE   

                                    titleDescription  \
0  Innovation Works aims to reduce Baltimore’s ne...   
1  Manage mergers & acquisitions, corporate and r...   
2  I work with talented women in global roles who...   
3  A coalition of organizations  and practitioner...   
4  As a Lead Mentor for the Conscious Venture Lab...   

  pastExperienceCompanyName pastExperienceCompanyUrl  \
0                                                      
1                                                      
2                                                      
3                                                      
4                                                      

  pastExperienceCompanyTitle pastExperienceDate pastExperienceDuration  
0                                                                       
1                                                                       
2                                                                       
3                                                                       
4                                                                       

[5 rows x 31 columns]

In [9]:
# Make sure profileUrl in the first column.
cols = df.columns.tolist()
if cols[0] != 'profileUrl':
    cols.remove('profileUrl')  # Remove the "profileURL" from its current position.
    cols = ['profileUrl'] + cols  # Add "profileURL" at the first column.
    df = df[cols] 
df.head()

profileUrl  \
0  https://www.linkedin.com/sales/lead/ACwAAAE1Yq...   
1  https://www.linkedin.com/sales/lead/ACwAAACRHS...   
2  https://www.linkedin.com/sales/lead/ACwAAAAQQk...   
3  https://www.linkedin.com/sales/lead/ACwAAADeWj...   
4  https://www.linkedin.com/sales/lead/ACwAAAAY9Y...   

                                               query  \
0  https://www.linkedin.com/sales/search/people?s...   
1  https://www.linkedin.com/sales/search/people?s...   
2  https://www.linkedin.com/sales/search/people?s...   
3  https://www.linkedin.com/sales/search/people?s...   
4  https://www.linkedin.com/sales/search/people?s...   

                  timestamp          fullName firstName   lastName  \
0  2023-07-19T18:02:26.835Z        Neil Davis      Neil      Davis   
1  2023-07-19T18:02:26.835Z      Michael Hess   Michael       Hess   
2  2023-07-19T18:02:26.835Z  Barrie Zucal, MS    Barrie  Zucal, MS   
3  2023-07-19T18:02:26.835Z      Leslie Traub    Leslie      Traub   
4  2023-07-19T18:02:26.836Z   Theresa Daytner   Theresa    Daytner   

                         companyName  \
0                   Innovation Works   
1  ABS Advisory Services Group, Inc.   
2                   Zucal Consulting   
3           Inclusion Allies Network   
4              Conscious Venture Lab   

                                               title companyId  \
0                                    Business Mentor  11787752   
1                                       Chairman/CEO             
2              Independent Mentor, Coach, Consultant             
3                              Co-Founder and Mentor             
4  Lead Mentor | Angel Investor | Executive Coach...   3356498   

                                        companyUrl  ...  \
0  https://www.linkedin.com/sales/company/11787752  ...   
1                                                   ...   
2                                                   ...   
3                                                   ...   
4   https://www.linkedin.com/sales/company/3356498  ...   

                                      vmid  \
0  ACwAAAE1YqQBZoy53iiEXT0Ol-GrNh_0U0dhvqM   
1  ACwAAACRHSQB6TCtuIxEce2hAb9lV3NM_GzNPc8   
2  ACwAAAAQQkoBFSrQYiDk3QVKcRprtPvRSXe7ajk   
3  ACwAAADeWjEBRuyhZkIF7X7fpy6uBoxiXLQWqLY   
4  ACwAAAAY9YQBq8q_bnRNDw_10z49i6IR2gD7RKQ   

                                  linkedInProfileUrl isPremium isOpenLink  \
0  https://www.linkedin.com/in/ACwAAAE1YqQBZoy53i...     FALSE      FALSE   
1  https://www.linkedin.com/in/ACwAAACRHSQB6TCtuI...     FALSE      FALSE   
2  https://www.linkedin.com/in/ACwAAAAQQkoBFSrQYi...     FALSE      FALSE   
3  https://www.linkedin.com/in/ACwAAADeWjEBRuyhZk...     FALSE      FALSE   
4  https://www.linkedin.com/in/ACwAAAAY9YQBq8q_bn...      TRUE      FALSE   

                                    titleDescription  \
0  Innovation Works aims to reduce Baltimore’s ne...   
1  Manage mergers & acquisitions, corporate and r...   
2  I work with talented women in global roles who...   
3  A coalition of organizations  and practitioner...   
4  As a Lead Mentor for the Conscious Venture Lab...   

  pastExperienceCompanyName pastExperienceCompanyUrl  \
0                                                      
1                                                      
2                                                      
3                                                      
4                                                      

  pastExperienceCompanyTitle pastExperienceDate pastExperienceDuration  
0                                                                       
1                                                                       
2                                                                       
3                                                                       
4                                                                       

[5 rows x 31 columns]

In [10]:
print(df.shape)

(82, 31)


# Save filtered data to gsheet

## Create a empty Google sheet

In [18]:
gapi = google_file_api.GoogleFileApi()

INFO  file_cache is only supported with oauth2client<4.0.0


In [24]:
name = 'SN_Search5_Yiyun'
# gdrive_folder : dict, the id and the name of the Google Drive folder.
gdrive_folder  = gapi.get_folder_id_by_name(name)

INFO  Found folder: {'id': '1XWNGDnJrVICHAe-6V2cnoSklZpk0APc_', 'name': 'SN_Search5_Yiyun'}


In [25]:
# if you want to use another folder id, please change the folder id manually.
# gdrive_folder  = {'id': '1XWNGDnJrVICHAe-6V2cnoSklZpk0APc_', 'name': 'SN_Search5_Yiyun'} 

In [27]:
"""
Create a new Google file (sheet or doc).

:param gfile_type: str, the type of the Google file ('sheet' or 'doc').
:param gfile_name: str, the name of the new Google file.
:param folder_id: str, the id of the Google Drive folder.
:param user: str, the email address of the user to share the Google file (Optional).
:return: None
"""
gfile_type = 'sheet'
gsheet_name = 'sn_search5.search_export.filtered.gsheet'
user = ''

In [29]:
gapi.create_empty_google_file(
    gfile_type = gfile_type,
    gfile_name = gsheet_name,
    gdrive_folder = gdrive_folder,
    user = user
)

INFO  file_cache is only supported with oauth2client<4.0.0
INFO  Created a new Google sheet 'sn_search5.search_export.filtered.gsheet'.
INFO  Move the new Google sheet 'sn_search5.search_export.filtered.gsheet' to the dir 'SN_Search5_Yiyun'
INFO  Finished creating the new Google sheet 'sn_search5.search_export.filtered.gsheet'.


## Save filtered data

In [30]:
# A Google sheet with this name should already exist on the drive.
spread2 = gspread_pandas.Spread(
    gsheet_name,
    sheet="Sheet1",
    create_sheet=True,
)
spread2.df_to_sheet(df, index=False)